RECOMMENDATION MODEL

In [43]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
# Mock data creation
def create_mock_data():
    users_data = "/content/sample_data/1st_train.csv"
    # "/content/sample_data/train_train.csv"
    applicants = pd.read_csv(users_data)

    jobs_data = "/content/sample_data/jobs_data.csv"
    companies = pd.read_csv(jobs_data)

    train_applicants = applicants
    test_data = "/content/sample_data/1st_test.csv"
    # "/content/sample_data/test_train.csv"
    test_applicants = pd.read_csv(test_data)

    return train_applicants, test_applicants, companies

In [6]:
# @title
# # Mock data creation
# def create_mock_data():
#     users_data = "/content/sample_data/rematch_train_candidate_field.csv"
#     applicants = pd.read_csv(users_data)

#     jobs_data = "/content/sample_data/jobs_data.csv"
#     companies = pd.read_csv(jobs_data)

#     # train_applicants = applicants
#     # test_data = "/content/sample_data/test_data_new.csv"
#     # test_applicants = pd.read_csv(test_data)

#     train_applicants = applicants[:10000]
#     test_applicants = applicants[10000:]

#     return train_applicants, test_applicants, companies

In [7]:
train_user, test_user, jobs = create_mock_data()

In [8]:
print("Training data size:", train_user.shape[0])
print("Test data size:", test_user.shape[0])

Training data size: 18979
Test data size: 4745


In [9]:
list_hard_skill = [test_user["hard_skill"].iloc[i].replace("[", "").replace("]", "").replace("'", "") for i in range(len(test_user))]
list_soft_skill = [test_user["soft_skill"].iloc[i].replace("[", "").replace("]", "").replace("'", "") for i in range(len(test_user))]

In [10]:
test_user["final_hard_skill"] = pd.DataFrame(list_hard_skill)
test_user["final_soft_skill"] = pd.DataFrame(list_soft_skill)
test_user.head(3)

,User ID,candidate_field,label,hard_skill,soft_skill,final_hard_skill,final_soft_skill
0,14649,it jobs,1,"['act', 'advertising sales', 'algorithms', 'bu...","['collaboration', 'decision making', 'operatio...","act, advertising sales, algorithms, business, ...","collaboration, decision making, operations, wr..."
1,801,marketing,0,"['act', 'brand communication', 'business', 'bu...","['collaboration', 'customer service', 'managem...","act, brand communication, business, business d...","collaboration, customer service, management"
2,4393,accounting,0,"['application', 'balance sheet', 'finance', 'p...","['filing', 'management']","application, balance sheet, finance, property ...","filing, management"


In [11]:
list_hard_skill = [train_user["hard_skill"].iloc[i].replace("[", "").replace("]", "").replace("'", "") for i in range(len(train_user))]
list_soft_skill = [train_user["soft_skill"].iloc[i].replace("[", "").replace("]", "").replace("'", "") for i in range(len(train_user))]

In [12]:
train_user["final_hard_skill"] = pd.DataFrame(list_hard_skill)
train_user["final_soft_skill"] = pd.DataFrame(list_soft_skill)
train_user.head(3)

,User ID,candidate_field,label,hard_skill,soft_skill,final_hard_skill,final_soft_skill
0,3030,sales,0,"['blogs', 'business', 'lead generation', 'mark...","['customer service', 'driven personality', 'ma...","blogs, business, lead generation, marketing st...","customer service, driven personality, manageme..."
1,9702,administration & office support,0,"['business', 'draft', 'go', 'manufacturing', '...","['business acumen', 'communications', 'managem...","business, draft, go, manufacturing, office man...","business acumen, communications, management, o..."
2,8606,retail & consumer products,0,"['gross profit', 'inventory', 'inventory manag...","['customer service', 'management']","gross profit, inventory, inventory management,...","customer service, management"


In [13]:
list_hard_skill = [jobs["Hard Skills"].iloc[i].replace("[", "").replace("]", "").replace("'", "") for i in range(len(jobs))]
list_soft_skill = [jobs["Soft Skills"].iloc[i].replace("[", "").replace("]", "").replace("'", "") for i in range(len(jobs))]

In [14]:
jobs["final_hard_skill"] = pd.DataFrame(list_hard_skill)
jobs["final_soft_skill"] = pd.DataFrame(list_soft_skill)
jobs.head(3)

,Job ID,Major,Hard Skills,Soft Skills,final_hard_skill,final_soft_skill
0,1,accounting,"['business', 'finance', 'excel', 'tax', 'servi...","['management', 'planning', 'operations', 'lead...","business, finance, excel, tax, service, data, ...","management, planning, operations, leadership, ..."
1,2,administration & office support,"['service', 'business', 'data', 'excel', 'appl...","['management', 'customer service', 'microsoft ...","service, business, data, excel, application, s...","management, customer service, microsoft office..."
2,3,"advertising, arts & media","['business', 'digital', 'sales', 'service', 'a...","['management', 'social media', 'writing', 'com...","business, digital, sales, service, application...","management, social media, writing, communicati..."


In [37]:
# Feature Engineering
def feature_engineering(applicants, companies):
    # Vectorize skills and majors
    tfidf_vectorizer_skills = TfidfVectorizer()
    tfidf_vectorizer_majors = TfidfVectorizer()

    all_skills = pd.concat([applicants['final_hard_skill'], applicants['final_soft_skill'],
                            companies['final_hard_skill'], companies['final_soft_skill']])
    all_majors = pd.concat([applicants['candidate_field'], companies['Major']])

    all_skills_vectorized = tfidf_vectorizer_skills.fit_transform(all_skills)
    all_majors_vectorized = tfidf_vectorizer_majors.fit_transform(all_majors)

    num_applicants = len(applicants)
    num_companies = len(companies)

    # Split the TF-IDF vectors back into applicants and companies
    applicants_skills_vectorized = all_skills_vectorized[:num_applicants*2]  # because each applicant has 2 skill entries
    companies_skills_vectorized = all_skills_vectorized[num_applicants*2:]

    applicants_majors_vectorized = all_majors_vectorized[:num_applicants]
    companies_majors_vectorized = all_majors_vectorized[num_applicants:]

    return (applicants_skills_vectorized, applicants_majors_vectorized,
            companies_skills_vectorized, companies_majors_vectorized, tfidf_vectorizer_skills, tfidf_vectorizer_majors)

In [38]:
def compute_similarity(applicants_skills_vectorized, applicants_majors_vectorized,
                       companies_skills_vectorized, companies_majors_vectorized):
    # Calculate similarity based on skills (averaging hard and soft skills similarities)
    applicants_skills = (applicants_skills_vectorized[0::2] + applicants_skills_vectorized[1::2]) / 2
    companies_skills = (companies_skills_vectorized[0::2] + companies_skills_vectorized[1::2]) / 2

    skills_similarity = cosine_similarity(applicants_skills, companies_skills)

    # Calculate similarity based on majors
    majors_similarity = cosine_similarity(applicants_majors_vectorized, companies_majors_vectorized)

    # Ensure the number of companies in both similarities is aligned
    if skills_similarity.shape[1] != majors_similarity.shape[1]:
        min_dim = min(skills_similarity.shape[1], majors_similarity.shape[1])
        skills_similarity = skills_similarity[:, :min_dim]
        majors_similarity = majors_similarity[:, :min_dim]

    # Combine these similarities (simple average for this example)
    combined_similarity = (skills_similarity + majors_similarity) / 2
    return combined_similarity

In [39]:
# Recommendation Function
def recommend_jobs(applicants, companies, similarity_scores):
    recommendations = {}
    for i, applicant in enumerate(applicants['User ID']):
        if i < len(similarity_scores):
            sorted_company_indices = np.argsort(-similarity_scores[i])  # Descending sort of scores
            recommended_companies = companies.iloc[sorted_company_indices]['Major'].values[:3]  # Top 3 recommendations
            recommendations[applicant] = recommended_companies
    return recommendations

# Testing and Evaluation Function
def print_recommendations(applicants, companies, recommendations):
    # This is a mock function since we don't have ground truth to compare to.
    # In a real scenario, we would compare against actual matches or use some form of feedback.
    print("Recommendations for each applicant:")
    for applicant in recommendations:
        print(f"{applicant}: {recommendations[applicant]}")

In [40]:
# Let's create and process the data, and compute recommendations
# train_applicants, test_applicants, companies = create_mock_data()
applicants_skills_vec, applicants_majors_vec, companies_skills_vec, companies_majors_vec, tfidf_vectorizer_skills, tfidf_vectorizer_majors = feature_engineering(train_user, jobs)

similarity_scores = compute_similarity(applicants_skills_vec, applicants_majors_vec, companies_skills_vec, companies_majors_vec)
recommendations = recommend_jobs(test_user, jobs, similarity_scores)

# Output the recommendations to observe the results
print_recommendations(test_user, jobs, recommendations)

Streaming output truncated to the last 5000 lines.
5046: ['marketing' 'advertising, arts & media' 'administration & office support']
2043: ['accounting' 'advertising, arts & media' 'marketing']
3611: ['accounting' 'advertising, arts & media' 'marketing']
6712: ['it jobs' 'healthcare & medical' 'recruitment consultancy jobs']
17086: ['recruitment consultancy jobs' 'it jobs' 'healthcare & medical']
20679: ['it jobs' 'recruitment consultancy jobs' 'advertising, arts & media']
16506: ['it jobs' 'recruitment consultancy jobs' 'marketing']
324: ['sales' 'accounting' 'marketing']
21541: ['accounting' 'advertising, arts & media' 'marketing']
5183: ['it jobs' 'recruitment consultancy jobs' 'marketing']
4234: ['recruitment consultancy jobs' 'it jobs' 'marketing']
22070: ['advertising, arts & media' 'marketing' 'healthcare & medical']
11773: ['it jobs' 'healthcare & medical' 'recruitment consultancy jobs']
8820: ['administration & office support' 'accounting'
 'advertising, arts & media']
14512: 

In [44]:
# Process input skills and recommend jobs
def recommend_jobs_for_input_skills(input_hard_skills, input_soft_skills, input_major, companies, tfidf_vectorizer_skills, tfidf_vectorizer_majors, companies_skills_vec, companies_majors_vec):
    input_hard_skills_vec = tfidf_vectorizer_skills.transform([input_hard_skills])
    input_soft_skills_vec = tfidf_vectorizer_skills.transform([input_soft_skills])
    input_major_vec = tfidf_vectorizer_majors.transform([input_major])

    # Average the vectorized hard and soft skills
    input_skills_vec = (input_hard_skills_vec + input_soft_skills_vec) / 2

    # Compute similarities
    skills_similarity = cosine_similarity(input_skills_vec, companies_skills_vec)
    major_similarity = cosine_similarity(input_major_vec, companies_majors_vec)

    # Ensure the number of companies in both similarities is aligned
    if skills_similarity.shape[1] != major_similarity.shape[1]:
        min_dim = min(skills_similarity.shape[1], major_similarity.shape[1])
        skills_similarity = skills_similarity[:, :min_dim]
        major_similarity = major_similarity[:, :min_dim]

    # Combine similarities
    combined_similarity = (skills_similarity + major_similarity) / 2

    # Get top 3 job recommendations
    sorted_company_indices = np.argsort(-combined_similarity[0])
    recommended_companies = companies.iloc[sorted_company_indices]['Major'].values[:3]

    return recommended_companies

"""TEST RECOMMENDED SYSTEM"""

input_hard_skills = "Business, Finance, Excel"
input_soft_skills = "Communication, Teamwork"
input_major = "Marketing"

recommended_jobs = recommend_jobs_for_input_skills(input_hard_skills, input_soft_skills, input_major, companies, tfidf_vectorizer_skills, tfidf_vectorizer_majors, companies_skills_vec, companies_majors_vec)
print("Recommended Jobs based on input skills and major:")
print(recommended_jobs)

Recommended Jobs based on input skills and major:
['marketing' 'sales' 'administration & office support']


TEST RECOMMENDED SYSTEM

Evaluating (PENDING)

In [ ]:
def create_ground_truth(csv_file_path):
    data = pd.read_csv(csv_file_path)

    # Tạo dictionary `ground_truth`
    ground_truth = {}
    for index, row in data.iterrows():
        user_id = row['User ID']
        actual_major = row['candidate_field']

        # Thêm vào dictionary, giả sử mỗi ứng viên chỉ chọn một công việc
        ground_truth[user_id] = [actual_major]

    return ground_truth

# Sử dụng hàm trên để tạo `ground_truth`
csv_file_path = '/content/sample_data/1st_test.csv'
ground_truth = create_ground_truth(csv_file_path)

In [ ]:
display(ground_truth)

{14649: ['it jobs'],
 801: ['marketing'],
 4393: ['accounting'],
 10045: ['marketing'],
 11347: ['accounting'],
 15999: ['it jobs'],
 15760: ['it jobs'],
 18080: ['it jobs'],
 20822: ['it jobs'],
 19869: ['it jobs'],
 11027: ['marketing'],
 15846: ['it jobs'],
 10849: ['recruitment consultancy jobs'],
 3153: ['sales'],
 11942: ['accounting'],
 2571: ['retail & consumer products'],
 4355: ['retail & consumer products'],
 10479: ['sales'],
 2755: ['administration & office support'],
 3970: ['retail & consumer products'],
 16927: ['it jobs'],
 4068: ['retail & consumer products'],
 15017: ['it jobs'],
 18881: ['it jobs'],
 12193: ['it jobs'],
 13513: ['recruitment consultancy jobs'],
 153: ['telecommunication'],
 1737: ['healthcare & medical'],
 18685: ['it jobs'],
 10942: ['retail & consumer products'],
 21187: ['it jobs'],
 21139: ['it jobs'],
 20363: ['it jobs'],
 249: ['telecommunication'],
 11218: ['marketing'],
 16430: ['it jobs'],
 14184: ['accounting'],
 9032: ['healthcare & medic

In [ ]:
def precision_at_k(recommendations, ground_truth, k=3):
    """
    Calculate the precision at k for recommendation system.

    Parameters:
    - recommendations (dict): Dictionary where keys are user IDs and values are lists of recommended majors.
    - ground_truth (dict): Dictionary where keys are user IDs and values are lists of truly suitable majors.
    - k (int): The number of top recommendations to consider for calculating precision.

    Returns:
    - float: The average precision at k for all users.
    """
    precision_scores = []

    for applicant, recommended_major in recommendations.items():
        if applicant in ground_truth:
            # Get top k recommendations
            top_k_recs = recommended_major[:k]
            # Calculate the number of relevant recommendations
            relevant_recs = sum(1 for major in top_k_recs if major in ground_truth[applicant])
            # Precision at k for this user
            precision = relevant_recs / k
            precision_scores.append(precision)

    # Average precision at k over all users
    average_precision = np.mean(precision_scores) if precision_scores else 0
    return average_precision

avg_precision = precision_at_k(recommendations, ground_truth)
print("Average Precision@3 with 18979 trains and 4745 tests:", avg_precision)

Average Precision@3 with 18979 trains and 4745 tests: 0.1252546540217773


In [ ]:
def recall_at_k(recommendations, ground_truth, k=3):
    recall_scores = []

    for user_id, recommended_majors in recommendations.items():
        if user_id in ground_truth:
            # Get top k recommendations
            top_k_recs = recommended_majors[:k]
            # Calculate the number of relevant recommendations
            relevant_recs = sum(1 for major in top_k_recs if major in ground_truth[user_id])
            # Calculate the total number of relevant items
            total_relevant = len(ground_truth[user_id])
            # Recall at k for this user
            recall = relevant_recs / total_relevant if total_relevant else 0
            recall_scores.append(recall)

    # Average recall at k over all users
    average_recall = sum(recall_scores) / len(recall_scores) if recall_scores else 0
    return average_recall

# Example usage:
avg_recall = recall_at_k(recommendations, ground_truth)
print("Average Recall@3 with 18979 trains and 4745 tests:", avg_recall)


Average Recall@3 with 18979 trains and 4745 tests: 0.3757639620653319


In [ ]:
def f1_score_at_k(recommendations, ground_truth, k=3):
    precision = precision_at_k(recommendations, ground_truth, k)
    recall = recall_at_k(recommendations, ground_truth, k)

    if precision + recall == 0:
        return 0

    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

avg_f1_score = f1_score_at_k(recommendations, ground_truth)

print("Average F1 Score@3:", avg_f1_score)

Average F1 Score@3: 0.18788198103266596
